In [1]:
import requests 

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

from time import sleep
import os
import csv
#from dotenv import load_dotenv
from pprint import pprint
import numpy as np

#load_dotenv()

# Guardar resultados en un csv de forma manual (no sobreescribe, añade a lo que haya)

Es importante que esta función así escrita ha de ejecutarse sobre un archivo que ya hayamos creado con las columnas que queremos, porque esta función solo mete datos, pero no le pone los nombres a las columnas. Para hacer eso podemos ejecutar una línea como la siguiente antes de hacer el scrapeo

In [4]:
'''ESTO BORRA LOS DATOS QUE HAYA EN EL CSV'''

columnas = ['nombre', 'numero de opiniones', 'puntuacion sobre 5', 'ranking', 'precio', 'tipos de cocina', 'dietas especiales', 'ubicaciones', 'puntuaciones desglosadas']
csv_name = 'un_nombre_nuevo.csv'

with open(csv_name, mode='w', newline='', encoding='utf-8') as csvfile: # abro el csv en modo escritura (borra lo que haya)
        writer = csv.writer(csvfile) # creo un objeto writer que permite escribir en el csv
        writer.writerow(columnas) # meto una fila que tenga el nombre de las columnas
        csvfile.close() # cierro el archivo

In [2]:
# El mode='a' hace que siga escribiendo si el archivo ya tiene algo dentro, en vez de sobreescribir el archivo
def guardar_csv(data, columnas, csv_name):
    with open(csv_name, mode='a', newline='', encoding='utf-8') as csvfile: # abro el csv en modo append
        writer = csv.writer(csvfile) # creo un objeto writer que permite escribir en el csv
        for i in range(len(data[0])): # creo las filas que tengo que ir metiendo en el csv (hay que ir fila por fila)
            row = [field_list[i] for field_list in data]
            writer.writerow(row)
        csvfile.close() # cierro el archivo

# Guardar la url de la página actual de restaurantes en un .txt por si el código falla 
(Sobreescribe)

In [3]:
def guardar_url_txt(browser, txt_name):
    with open(txt_name, 'w') as txtfile: # abro el txt
        txtfile.write(browser.current_url) # escribo la url actual
        txtfile.close() # cierro el txt

# Sacar la url del archivo .txt

In [4]:
def sacar_url_txt(txt_name): 
    with open(txt_name, 'r') as txtfile: # abro el txt en modo 'r', de lectura
        return txtfile.read() # retorno lo que arroja la lectura (la url)

# WebScrapping a TripAdvisor con Selenium

In [ ]:
# Abrimos Chrome con TripAdvisor
browser = webdriver.Chrome()
url = sacar_url_txt('current_url_ourense.txt') # saco la url del txt
browser.get(url)

sleep(2)

# Aceptamos cookies
browser.find_element(by = By.ID, value = 'onetrust-accept-btn-handler').click()

sleep(2)




# Esta lista es auxiliar para conseguir las puntuaciones despues
tipos_rating = ['Pésimo', 'Malo', 'Normal', 'Muy bueno', 'Excelente']
columnas = ['nombre', 'numero de opiniones', 'puntuacion sobre 5', 'ranking', 'precio', 'tipos de cocina', 'dietas especiales', 'ubicaciones', 'puntuaciones desglosadas']


while True:
    # Creamos listas vacias para rellenar y crear el DataFrame
    tipos_cocina = []
    puntuacion_sobre5 = []
    ranking = []
    precios = []
    puntuaciones_desglosadas = []
    num_opiniones = []
    dietas_especiales = []
    ubicaciones = []
    nombres = []
    comidas = []
    ventajas = []

    # Obtenemos la lista de 30 restaurantes que muestra la primera pagina
    sleep(2)
    acceso_restaurantes = browser.find_elements(By.CSS_SELECTOR, '.biGQs._P.fiohW.alXOW.NwcxK.GzNcM.ytVPx.UTQMg.RnEEZ.ngXxk')

    # Guardamos la ventana de esta primera pagina para volver despues
    original_window = browser.current_window_handle
    
    # Recorremos los restaurantes y extraemos sus datos
    for rest in acceso_restaurantes:
        sleep(2)
        rest.click()
        
        for window_handle in browser.window_handles:
            if window_handle != original_window:
                browser.switch_to.window(window_handle)
                break
                
        # Sacamos la sopa de la pagina del restaurante
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
        try:
            puntuacion_sobre5.append(soup.find('span', class_ = 'ZDEqb').text[:3])
        except:
            puntuacion_sobre5.append(np.nan)
            print('falla puntuacion_sobre5')
        try:
            num_opiniones.append(soup.find('a', class_ = 'IcelI').text.split()[0])
        except:
            num_opiniones.append(np.nan)
            print('falla num_opiniones')
        try:
            ubicaciones.append(soup.find('span', class_ = 'yEWoV').text)
        except:
            ubicaciones.append(np.nan)
            print('falla ubicaciones')
        try:
            nombres.append(soup.find('h1', 'HjBfq').text)
        except:
            nombres.append(np.nan)
            print('falla nombres')
        try:
            lista_temp = []
            for idx, tipo_rating in enumerate(tipos_rating):
                lista_temp.append(soup.find('div', {'data-value': f'{idx+1}'}).text.strip(tipo_rating))
            puntuaciones_desglosadas.append(lista_temp)
        except:
            puntuaciones_desglosadas.append(np.nan)
            print('falla puntuaciones_desglosadas')
        
        # Los detalles pueden ser de dos tipos dependiendo de si ofrecen informacion del precio o no.
        detalles = soup.find_all('div', class_ = 'SrqKb')
        if not detalles:
            print('No hay detalles')
            comidas.append(np.nan)
            ventajas.append(np.nan)
            precios.append(np.nan)
            dietas_especiales.append(np.nan)
            tipos_cocina.append(np.nan)
        else:  
            if '€' in detalles[0].text:
                try:
                    precios.append(detalles[0].text.split()[::3])
                except:
                    precios.append(np.nan)
                    print('falla precios')
                try:
                    tipos_cocina.append(detalles[1].text.split(','))
                except:
                    tipos_cocina.append(np.nan)
                    print('falla tipos_cocina')
                try:
                    dietas_especiales.append(detalles[2].text.split(','))
                except:
                    dietas_especiales.append(np.nan)
                    print('falla dietas_especiales')     
                comidas.append(np.nan)
                ventajas.append(np.nan)
            else:                 
                try:
                    tipos_cocina.append(detalles[0].text.split(','))
                except:
                    tipos_cocina.append(np.nan)
                    print('falla precios')
                try:
                    comidas.append(detalles[1].text.split(','))
                except:
                    comidas.append(np.nan)
                    print('falla tipos_cocina')
                try:
                    ventajas.append(detalles[2].text.split(','))
                except:
                    ventajas.append(np.nan)
                    print('falla dietas_especiales')     
                precios.append(np.nan)
                dietas_especiales.append(np.nan)
        try:
            ranking.append(soup.find('b').text[1:])
        except:
            ranking.append(np.nan)
            print('falla ranking')

        # cerramos pestaña y volvemos a la ventana original
        sleep(4)
        if browser.current_window_handle != original_window:
            browser.close()
            browser.switch_to.window(original_window)
            
    
    # Guardamos los datos    
    try:      
        data = [nombres, num_opiniones, puntuacion_sobre5, ranking, precios, tipos_cocina, dietas_especiales, ubicaciones, puntuaciones_desglosadas]
        guardar_csv(data, columnas, 'datos_ourense.csv')
        guardar_url_txt(browser, 'current_url_ourense.txt')
    except:
        print('No se han podido guardar los datos')
        
    # Vamos a la siguiente pagina
    try:
        sleep(3)
        browser.find_element(By.CSS_SELECTOR, '[data-smoke-attr=pagination-next-arrow]').click()
    except:
        print('Hemos acabado')
        break

falla nombres
falla nombres
falla dietas_especiales
falla puntuacion_sobre5
falla num_opiniones
falla ubicaciones
falla nombres
falla puntuaciones_desglosadas
No hay detalles
falla ranking
falla dietas_especiales
falla dietas_especiales
falla tipos_cocina
falla dietas_especiales
falla puntuacion_sobre5
falla num_opiniones
falla ubicaciones
falla nombres
falla puntuaciones_desglosadas
No hay detalles
falla ranking
falla dietas_especiales
falla nombres
falla dietas_especiales
falla dietas_especiales
falla tipos_cocina
falla dietas_especiales
No hay detalles
falla dietas_especiales
falla tipos_cocina
falla dietas_especiales
falla tipos_cocina
falla dietas_especiales
falla dietas_especiales
falla tipos_cocina
falla dietas_especiales
No hay detalles
falla tipos_cocina
falla dietas_especiales
falla dietas_especiales
falla tipos_cocina
falla dietas_especiales
No hay detalles
No hay detalles
falla tipos_cocina
falla dietas_especiales
No hay detalles
falla tipos_cocina
falla dietas_especiales
f

# Pasando direcciones a coordenadas con Google Geocoding API

In [135]:
def obtener_coordenadas(direccion):
    API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
    BASE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': direccion,
        'key': API_KEY
    }
    respuesta = requests.get(BASE_URL, params=params)
    datos = respuesta.json()
    if datos['status'] == 'OK':
        ubicacion = datos['results'][0]['geometry']['location']
        return ubicacion['lat'], ubicacion['lng']
    else:
        print('Error:', datos['status'])
        return None
direccion = '1600 Amphitheatre Parkway, Mountain View, CA'
latitud, longitud = obtener_coordenadas(direccion)
print(f'Latitud: {latitud}, Longitud: {longitud}')

Latitud: 37.4223878, Longitud: -122.0841877


# Basura

In [7]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [8]:
nombres = [x.text for x in soup.find_all('div', class_ = 'result-title')]

In [23]:
titulos = soup.find_all('div', class_ = 'result-title')
nombres = [x.text for x in titulos]
urls = ['https://www.tripadvisor.es'+ x['onclick'].split()[3].strip("',") for x in titulos]

In [9]:
soup_reviews = soup.find_all('div', class_ = 'rating-review-count')
cantidad_burbujas = [x.find('span')['alt'] for x in soup_reviews]
cantidad_opiniones = [x.text for x in soup_reviews]

In [10]:
direcciones = [x.text for x in soup.find_all('div', class_ = 'address-text')]

In [26]:
df = pd.DataFrame([nombres, cantidad_burbujas, cantidad_opiniones, direcciones, urls], ['nombre', 'burbujas', 'opiniones', 'direccion', 'url']).T

In [15]:
data = [nombres, num_opiniones, puntuacion_sobre5, ranking, precios, tipos_cocina, dietas_especiales, ubicaciones, puntuaciones_desglosadas] 
columnas = ['nombre', 'numero de opiniones', 'puntuacion sobre 5', 'ranking', 'precio', 'tipos de cocina', 'dietas especiales', 'ubicaciones', 'puntuaciones desglosadas']
df = pd.DataFrame(data, columnas).T
df

,nombre,numero de opiniones,puntuacion sobre 5,ranking,precio,tipos de cocina,dietas especiales,ubicaciones,puntuaciones desglosadas
0,Los Montes de Galicia,11.361,"5,0",1,"[50, 90]","[Española,, Internacional,, Mediterránea,, Sal...","[Opciones veganas, Opciones sin gluten]","Calle Azcona 46, 28028 Madrid España","[7, 28, 175, 1.082, 7.394]"
1,Trattoria Pulcinella,2766,"4,5",15,"[11, 29]","[Italiana,, Napolitana,, De, Campania,, Romana...","[Opciones vegetarianas, Opciones veganas, Op...","C/ de Regueros, 7, 28004 Madrid España","[34, 39, 104, 320, 1.620]"
2,GastroVía 61,2018,"4,5",176,"[30, 45]","[Mediterránea,, Europea,, Española,, Saludable]","[Opciones vegetarianas, Opciones sin gluten]","Calle Flor Baja, 3 Hotel Mayorazgo, 28013 Madr...","[8, 13, 45, 155, 1.296]"
3,Chocolatería San Ginés,23.394,"4,0",.º 9,"[2, 6]","[Café,, Española]",[Abierto hasta tarde],"Pasadizo de San Ginés 5, 28013 Madrid España","[479, 603, 1.840, 4.195, 6.252]"


In [111]:
for idx, tipo_rating in enumerate(tipos_rating):
            lista_temp.append(soup.find('div', {'data-value': f'{idx+1}'}).text.strip('tipo_rating'))
        puntuaciones_desglosadas.append(lista_temp)

'Bar La Campana'

In [10]:
soup.find('div', {'data-value': '5'}).text

'Excelente7.394'

In [95]:
soup.find('h1', 'HjBfq').text

'Los Montes de Galicia'

In [71]:
soup.find('b').text[1:]

'1'

In [91]:
tipos_rating = ['Excelente', 'Muy bueno', 'Normal', 'Malo', 'Pésimo']
tipos_rating = tipos_rating[::-1]

In [92]:
tipos_rating

['Pésimo', 'Malo', 'Normal', 'Muy bueno', 'Excelente']

In [14]:
def guardar_csv(data, columnas, csv_name):
    with open(csv_name, mode='a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(columnas)
        for i in range(len(data[0])):
            row = [field_list[i] for field_list in data]
            writer.writerow(row)
        csvfile.close()

In [15]:
guardar_csv(data, columnas, 'datos_tripadvisor3.csv')

In [13]:
data = [nombres, num_opiniones, puntuacion_sobre5, ranking, precios, tipos_cocina, dietas_especiales, ubicaciones, puntuaciones_desglosadas]
